# Price and Markdown Optimization for Multiple Products

This notebook demonstrates how to optimize prices or markdowns for multiple related products such as substitutable products within one category.

### Use Case
We consider a scenario where a seller offers multiple products in a category or group, so that the products are fully or partly substitutable. We assume that each product has its own price-demand function which is dependent on all other products. Our goal is to determine the revenue/profit-optimal price for each product taking into account the cross-dependencies.

### Prototype: Approach and Data
In the general case, the demand function for each product depends on all individual prices of other products. This brings a layer of complexity to accurate estimation and optimization, especially in the dynamic pricing settings. One possible simplification is to use a demand function that depends not on the individual prices of other products, but on the average price within a group of substitutable products. This can be an accurate approximation in many cases, because the ratio between a product’s own price and the average price in the group reflects the competitiveness of the product and quantifies demand cannibalization. In this case, we can assume a demand model that estimates multiple values for each possible average price instead of separate demand values for each product-price pair (the set of possible average prices is finite because the set of valid price levels is discrete). 

This concept can naturally be implemented using integer programming (IP). We formulate the optimization problem in linear programming (LP) terms for the sake of scalability (technique known as *linear relaxation*).

### Usage and Productization
The prototype uses the LP-based approach which provides enough scalability for optimizing large sets of products. Productization would requre to specify price-demand functions that account for the average price.

### References
1. Ferreira K., Simchi-Levi D., and Wang H. -- Online Network Revenue Management Using Thompson Sampling, November 2017

In [1]:
#
# Imports and settings
#
import numpy as np
from tabulate import tabulate
from scipy.optimize import linprog

def tabprint(msg, A):
    print(msg)
    print(tabulate(A, tablefmt="fancy_grid"))

# Markdown Optimization as an LP Problem

We solve the following optimization problem:

$\text{max} \quad \sum_{i=1}^{N} \sum_{k=1}^{K} z_{ik} \cdot p_{k} \cdot q\left(i, p_{k}, c\right)$ 

$\text{subject to}$

$\quad \sum_{i=1}^{N} \sum_{k=1}^{K} z_{ik} \cdot q\left(i, p_{k}, c\right) = c$

$\quad \sum_{k=1}^{N} z_{ik} = 1, \quad \text{for}\ i=1,\ldots,N $

$\quad z_{ik}\ge 0 $

where q is demand, p is price, N is the number products, K is the number of price levels, and c is the sum of prices (the measure of the average price). The demand values are assumed to be computed for a specific values of $c$. 

In [9]:
# prices - k-dimensional vector of allowed price levels 
# demands - n x k matrix of demand predictions
# c - required sum of prices 
# where k is the number of price levels, n is number of products
def optimal_prices_category(prices, demands, c):
    
    n, k = np.shape(demands)  

    # prepare inputs   
    r = np.multiply(np.tile(prices, n), np.array(demands).reshape(1, k*n))
    A = np.array([[
        1 if j >= k*(i) and j < k*(i+1) else 0
        for j in range(k*n)
    ] for i in range(n)])
    A = np.append(A, np.tile(prices, n), axis=0)
    b = [np.append(np.ones(n), c)]

    # solve the linear program
    res = linprog(-r.flatten(), 
              A_eq = A, 
              b_eq = b,  
              bounds=(0, 1))

    return np.array(res.x).reshape(n, k)

In [10]:
#
# Test run
#
r = optimal_prices_category(
  np.array([[1.00, 1.50, 2.00, 2.50]]),   # allowed price levels
  np.array([
    [ 28, 23, 20, 13],                    # demands for product 1
    [ 30, 22, 16, 12],                    # demands for product 2
    [ 32, 26, 19, 15]                     # demands for product 3
  ]), 5.50)                               # sum of prices for all three products

In [13]:
tabprint('prices = ', r)

prices = 
╒═════════════╤═════════════╤═════════════╤═════════════╕
│ 3.53009e-11 │ 2.68242e-10 │ 1           │ 3.29004e-10 │
├─────────────┼─────────────┼─────────────┼─────────────┤
│ 6.92591e-11 │ 1           │ 3.21738e-10 │ 2.62897e-10 │
├─────────────┼─────────────┼─────────────┼─────────────┤
│ 2.4468e-10  │ 0.5         │ 1.29104e-09 │ 0.5         │
╘═════════════╧═════════════╧═════════════╧═════════════╛


In [15]:
# output
# [[0  0     1   0  ]                            # price vector for product 1 ($2.00)
#  [0  1     0   0  ]                            # price vector for product 2 ($1.50)
#  [0  0.5   0   0.5]]                           # price vector for product 3 ($1.50 and 2.50)